In [1]:
!pip -q install transformers==4.17
!pip -q install optuna
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 20.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.8 M

In [3]:
from transformers import Trainer, TrainingArguments, AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import torch
import pandas as pd
import nltk
import re
import ast
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
import torch.nn as nn
import optuna
import time

In [5]:
splits = {'java_train': 'data/java_train-00000-of-00001.parquet', 'java_test': 'data/java_test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["java_train"])
test_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["java_test"])

display(train_df.head())
display(test_df.head())
print(train_df.shape)

,index,class,comment_sentence,partition,combo,labels
0,0,Abfss.java,azure blob file system implementation of abstr...,0,azure blob file system implementation of abstr...,"[1, 0, 0, 0, 0, 0, 0]"
1,1,Abfss.java,this impl delegates to the old filesystem,0,this impl delegates to the old filesystem | Ab...,"[0, 0, 1, 0, 0, 0, 0]"
2,2,AbstractContractGetFileStatusTest.java,test getfilestatus and related listing operati...,0,test getfilestatus and related listing operati...,"[1, 0, 0, 0, 0, 0, 0]"
3,11,AbstractContractGetFileStatusTest.java,path filter which only expects paths whose fin...,0,path filter which only expects paths whose fin...,"[0, 0, 1, 0, 0, 0, 0]"
4,15,AbstractContractGetFileStatusTest.java,a filesystem filter which exposes the protecte...,0,a filesystem filter which exposes the protecte...,"[0, 0, 1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,5,AbstractContractGetFileStatusTest.java,accept everything.,1,accept everything. | AbstractContractGetFileSt...,"[0, 0, 1, 0, 0, 0, 0]"
1,8,AbstractContractGetFileStatusTest.java,accept nothing.,1,accept nothing. | AbstractContractGetFileStatu...,"[0, 0, 1, 0, 0, 0, 0]"
2,12,AbstractContractGetFileStatusTest.java,equals the @code match field.,1,equals the @code match field. | AbstractContra...,"[0, 0, 1, 0, 0, 0, 0]"
3,17,AbstractS3ACommitterFactory.java,dynamically create the output committer based ...,1,dynamically create the output committer based ...,"[0, 0, 1, 0, 0, 0, 0]"
4,19,ApplicationConstants.java,environment for applications.,1,environment for applications. | ApplicationCon...,"[0, 0, 1, 0, 0, 0, 0]"


(7614, 6)


In [6]:
def remove_punctuation_except_pipe(text):
    return re.sub(r"[^\w\s\|]", "", text)

# Apply the function to the 'combo' column
train_df['combo'] = train_df['combo'].apply(remove_punctuation_except_pipe)
test_df['combo'] = test_df['combo'].apply(remove_punctuation_except_pipe)

# Convert data to lowercase
train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)
test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)

train_df[["class", "comment_sentence", "combo"]] = train_str_df[["class", "comment_sentence", "combo"]]
test_df[["class", "comment_sentence", "combo"]] = test_str_df[["class", "comment_sentence", "combo"]]

display(train_df.head())
display(test_df.head())

<ipython-input-6-df4392ae2e99>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
<ipython-input-6-df4392ae2e99>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(


,index,class,comment_sentence,partition,combo,labels
0,0,abfss.java,azure blob file system implementation of abstr...,0,azure blob file system implementation of abstr...,"[1, 0, 0, 0, 0, 0, 0]"
1,1,abfss.java,this impl delegates to the old filesystem,0,this impl delegates to the old filesystem | ab...,"[0, 0, 1, 0, 0, 0, 0]"
2,2,abstractcontractgetfilestatustest.java,test getfilestatus and related listing operati...,0,test getfilestatus and related listing operati...,"[1, 0, 0, 0, 0, 0, 0]"
3,11,abstractcontractgetfilestatustest.java,path filter which only expects paths whose fin...,0,path filter which only expects paths whose fin...,"[0, 0, 1, 0, 0, 0, 0]"
4,15,abstractcontractgetfilestatustest.java,a filesystem filter which exposes the protecte...,0,a filesystem filter which exposes the protecte...,"[0, 0, 1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,5,abstractcontractgetfilestatustest.java,accept everything.,1,accept everything | abstractcontractgetfilesta...,"[0, 0, 1, 0, 0, 0, 0]"
1,8,abstractcontractgetfilestatustest.java,accept nothing.,1,accept nothing | abstractcontractgetfilestatus...,"[0, 0, 1, 0, 0, 0, 0]"
2,12,abstractcontractgetfilestatustest.java,equals the @code match field.,1,equals the code match field | abstractcontract...,"[0, 0, 1, 0, 0, 0, 0]"
3,17,abstracts3acommitterfactory.java,dynamically create the output committer based ...,1,dynamically create the output committer based ...,"[0, 0, 1, 0, 0, 0, 0]"
4,19,applicationconstants.java,environment for applications.,1,environment for applications | applicationcons...,"[0, 0, 1, 0, 0, 0, 0]"


In [7]:
X = list(train_df['combo'])
y = list(train_df['labels'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [8]:
tokenizer = AutoTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')
model = AutoModelForSequenceClassification.from_pretrained('huawei-noah/TinyBERT_General_4L_312D', num_labels=7)

Downloading:   0%|          | 0.00/409 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/59.8M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

In [9]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, device="cpu"):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoded_text = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_text['input_ids'].squeeze().to(self.device)
        attention_mask = encoded_text['attention_mask'].squeeze().to(self.device)
        label = torch.tensor(label, dtype=torch.float).to(self.device)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

train_dataset = TextClassificationDataset(X_train, y_train, tokenizer)
eval_dataset = TextClassificationDataset(X_val, y_val, tokenizer)

In [10]:
class OneHotTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        model_inputs = {k: v for k, v in inputs.items() if k != "labels"}
        outputs = model(**model_inputs)
        logits = outputs.get("logits")

        if logits is None:
            raise ValueError("Logits are missing from the model output.")

        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())

        # Log training loss
        if self.state.global_step % self.args.logging_steps == 0:
            print(f"Step {self.state.global_step}, Training Loss: {loss.item()}")

        return (loss, outputs) if return_outputs else loss


# class OneHotTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         labels = inputs.get("labels")
#         # Remove 'labels' from inputs before passing to model
#         model_inputs = {k: v for k, v in inputs.items() if k != "labels"}
#         outputs = model(**model_inputs)  # Pass only the required inputs to the model
#         logits = outputs.get("logits")

#         if logits is None:
#             # Handle the case where logits are missing, e.g., raise an exception or return a default loss
#             raise ValueError("Logits are missing from the model output.")  # Or return a default loss

#         # Use BCEWithLogitsLoss for multi-label classification
#         loss_fct = nn.BCEWithLogitsLoss()
#         loss = loss_fct(logits, labels.float())

#         return (loss, outputs) if return_outputs else loss

In [11]:
def compute_metrics(pred):
    logits = pred.predictions
    preds = (logits > 0).astype(int)
    labels = pred.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=1)
    accuracy = accuracy_score(labels, preds)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


# def compute_metrics(pred):
#     logits = pred.predictions
#     preds = (logits > 0).astype(int)

#     labels = pred.label_ids

#     precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
#     accuracy = (preds == labels).mean()

#     return {
#         'accuracy': accuracy,
#         'precision': precision,
#         'recall': recall,
#         'f1': f1
#     }


In [12]:
def objective(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)

    # training_args = TrainingArguments(
    #     output_dir="./results",
    #     num_train_epochs=num_train_epochs,
    #     per_device_train_batch_size=batch_size,
    #     per_device_eval_batch_size=batch_size * 2,
    #     warmup_steps=10,
    #     weight_decay=weight_decay,
    #     evaluation_strategy="epoch",
    #     learning_rate=learning_rate,
    #     gradient_accumulation_steps=2,
    #     report_to="none",
    # )

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size * 2,
        warmup_steps=10,
        weight_decay=weight_decay,
        evaluation_strategy="epoch",
        logging_dir="./logs",  # Directory for logging
        logging_steps=10,  # Log every 10 steps
        save_steps=500,
        learning_rate=learning_rate,
        gradient_accumulation_steps=2,
        report_to="none",  # Avoid reporting to third-party loggers like WandB
        logging_first_step=True,  # Log the first step
    )

    trainer = OneHotTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )
    print("Starting training...")
    start_time = time.time()
    trainer.train()
    end_time = time.time()
    training_duration = end_time - start_time
    print(f"Training completed in {training_duration:.2f} seconds.")

    eval_results = trainer.evaluate()

    return eval_results["eval_f1"]

In [13]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print("<<=================Best trial=========================>>")
print(study.best_trial)
print("<<==========================================>>")

[I 2024-12-09 18:49:34,941] A new study created in memory with name: no-name-62a26867-41d7-4419-95be-4b93d57f058a
<ipython-input-12-42465065092b>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-12-42465065092b>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)


Starting training...


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6091
  Num Epochs = 6
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 2286


Step 0, Training Loss: 0.6944229006767273
Step 0, Training Loss: 0.6944547891616821


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.337000,0.333940,0.000000,1.000000,0.000000,0.000000


...

***** Running Evaluation *****
  Num examples = 1523
  Batch size = 8



Step 7610, Training Loss: 0.3552511930465698


Step 7610, Training Loss: 0.47686871886253357
Step 7610, Training Loss: 0.06700902432203293
Step 7610, Training Loss: 0.00841838400810957
Step 7610, Training Loss: 0.3608073592185974
Step 7610, Training Loss: 0.5022972226142883
Step 7610, Training Loss: 0.35146868228912354
Step 7610, Training Loss: 0.176463320851326
Step 7610, Training Loss: 0.010867771692574024
Step 7610, Training Loss: 0.004802650772035122
Step 7610, Training Loss: 0.2342839241027832
Step 7610, Training Loss: 0.19724126160144806
Step 7610, Training Loss: 0.22260023653507233
Step 7610, Training Loss: 0.18225494027137756
Step 7610, Training Loss: 0.19937878847122192
Step 7610, Training Loss: 0.11776187270879745
Step 7610, Training Loss: 0.22781188786029816
Step 7610, Training Loss: 0.11971457302570343
Step 7610, Training Loss: 0.26536378264427185
Step 7610, Training Loss: 0.1204332485795021
Step 7610, Training Loss: 0.22531473636627197
Step 7610, Training Loss: 0.001597349182702601
Step 7610, Training Loss: 0.358829319

[I 2024-12-09 19:36:32,385] Trial 9 finished with value: 0.8629014393184927 and parameters: {'learning_rate': 1.37587077953449e-05, 'batch_size': 4, 'num_train_epochs': 10, 'weight_decay': 0.00017332012008881235}. Best is trial 9 with value: 0.8629014393184927.


<<=================Best trial=========================>>
FrozenTrial(number=9, state=TrialState.COMPLETE, values=[0.8629014393184927], datetime_start=datetime.datetime(2024, 12, 9, 19, 29, 41, 256485), datetime_complete=datetime.datetime(2024, 12, 9, 19, 36, 32, 385568), params={'learning_rate': 1.37587077953449e-05, 'batch_size': 4, 'num_train_epochs': 10, 'weight_decay': 0.00017332012008881235}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=5e-05, log=True, low=1e-06, step=None), 'batch_size': CategoricalDistribution(choices=(4, 8, 16)), 'num_train_epochs': IntDistribution(high=10, log=False, low=3, step=1), 'weight_decay': FloatDistribution(high=0.1, log=True, low=0.0001, step=None)}, trial_id=9, value=None)
<<==========================================>>


In [14]:
best_hyperparams = study.best_trial.params

training_args = TrainingArguments(
    output_dir="./best_results",
    num_train_epochs=best_hyperparams["num_train_epochs"],
    per_device_train_batch_size=best_hyperparams["batch_size"],
    per_device_eval_batch_size=best_hyperparams["batch_size"] * 2,
    learning_rate=best_hyperparams["learning_rate"],
    weight_decay=best_hyperparams["weight_decay"],
    warmup_steps=10,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=2,
    report_to="none",
)

trainer = OneHotTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# Measure training time
print("Starting training...")
start_time = time.time()  # Start the timer

trainer.train()

end_time = time.time()  # End the timer
training_duration = end_time - start_time  # Calculate the duration
print("=====================Training Time=====================")
print(f"Training completed in {training_duration:.2f} seconds.")
print("=======================================================")

PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6091
  Num Epochs = 10
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 7610


Starting training...
Step 0, Training Loss: 0.001971847377717495
Step 0, Training Loss: 0.0015596641460433602


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,0.014600,0.190941,0.856861,0.867664,0.860719,0.859752
1,0.016000,0.191745,0.848982,0.861417,0.858151,0.856890
2,0.012900,0.191247,0.839133,0.860688,0.858793,0.859468
3,0.013000,0.202272,0.854235,0.859486,0.870347,0.862239
4,0.009100,0.200534,0.845699,0.862561,0.856226,0.857833
5,0.009500,0.204698,0.855548,0.867875,0.868421,0.866589
6,0.012200,0.204138,0.856861,0.861963,0.869705,0.863818
7,0.009500,0.211037,0.847669,0.860111,0.862003,0.859989
8,0.009500,0.203029,0.861458,0.865921,0.875481,0.869614
9,0.006900,0.205522,0.857518,0.863561,0.874198,0.867643


Saving model checkpoint to ./best_results/checkpoint-500
Configuration saved in ./best_results/checkpoint-500/config.json
Model weights saved in ./best_results/checkpoint-500/pytorch_model.bin


...

Step 7000, Training Loss: 0.013997846283018589
Step 7000, Training Loss: 0.0014883126132190228


Saving model checkpoint to ./best_results/checkpoint-7500
Configuration saved in ./best_results/checkpoint-7500/config.json
Model weights saved in ./best_results/checkpoint-7500/pytorch_model.bin


Step 7500, Training Loss: 0.001515594543889165
Step 7500, Training Loss: 0.0004796756256837398


***** Running Evaluation *****
  Num examples = 1523
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




=====================Training Time=====================
Training completed in 395.38 seconds.


In [ ]:
eval_results = trainer.evaluate()

### Model Saving

In [15]:
!mkdir 'best_model_huawei-noah_TinyBERT_General_4L_312D_java'
!mkdir 'best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer'

# Save model and tokenizer
trainer.save_model('./best_model_huawei-noah_TinyBERT_General_4L_312D_java')
tokenizer.save_pretrained('./best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer')

# Load model and tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('./best_model_huawei-noah_TinyBERT_General_4L_312D_java')
tokenizer = AutoTokenizer.from_pretrained('./best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer')

Saving model checkpoint to ./best_model_huawei-noah_TinyBERT_General_4L_312D_java
Configuration saved in ./best_model_huawei-noah_TinyBERT_General_4L_312D_java/config.json
Model weights saved in ./best_model_huawei-noah_TinyBERT_General_4L_312D_java/pytorch_model.bin
tokenizer config file saved in ./best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer/tokenizer_config.json
Special tokens file saved in ./best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer/special_tokens_map.json
loading configuration file ./best_model_huawei-noah_TinyBERT_General_4L_312D_java/config.json
Model config BertConfig {
  "_name_or_path": "./best_model_huawei-noah_TinyBERT_General_4L_312D_java",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "classifier_dropout": null,
  "emb_size": 312,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
   

### Model Testing

In [16]:
X_test = list(test_df['combo'])
y_test = list(test_df['labels'])

In [17]:
df_test = pd.DataFrame({"combo":X_test,"labels":y_test})
test_text = df_test.combo.values
test_label = df_test.labels.values

print(test_label)

[array([0, 0, 1, 0, 0, 0, 0]) array([0, 0, 1, 0, 0, 0, 0])
 array([0, 0, 1, 0, 0, 0, 0]) ... array([0, 1, 0, 0, 0, 0, 0])
 array([0, 1, 0, 0, 0, 0, 0]) array([0, 1, 0, 0, 0, 0, 0])]


In [18]:
test_dataset = TextClassificationDataset(test_text, test_label, tokenizer)

# Measure inference time
start_time = time.time()

# Predict on the test dataset
predictions = trainer.predict(test_dataset)

# Calculate elapsed time
end_time = time.time()
inference_time = end_time - start_time

# Compute metrics using the `compute_metrics` function
metrics = compute_metrics(predictions)

print("=================Evaluation Metrics================")
print("Evaluation Metrics:", metrics)
print("===================================================")

print("=================Inference Time================")
print(f"Inference Time: {inference_time:.2f} seconds")
print("===================================================")

***** Running Prediction *****
  Num examples = 1725
  Batch size = 8


=================Evaluation Metrics================
Evaluation Metrics: {'accuracy': 0.768695652173913, 'precision': 0.7916460568726307, 'recall': 0.7829591249280369, 'f1': 0.7822260983692482}
=================Inference Time================
Inference Time: 3.43 seconds


In [19]:
!zip -r best_model_huawei-noah_TinyBERT_General_4L_312D_java.zip './best_model_huawei-noah_TinyBERT_General_4L_312D_java'
!zip -r best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer.zip './best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer'

  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_java/ (stored 0%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_java/training_args.bin (deflated 51%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_java/pytorch_model.bin (deflated 7%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_java/config.json (deflated 54%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer/ (stored 0%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer/tokenizer_config.json (deflated 41%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer/vocab.txt (deflated 53%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer/special_tokens_map.json (deflated 40%)
  adding: best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer/tokenizer.json (deflated 71%)


In [21]:
# Transferring the model to Google Drive
from google.colab import drive
drive.mount('/content/drive')

!cp best_model_huawei-noah_TinyBERT_General_4L_312D_java.zip "/content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_java/"
!cp best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer.zip "/content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_java_Tokenizer/"

Mounted at /content/drive


### Load and Test Model

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Replace 'path/to/checkpoint-folder' with the actual path to your checkpoint folder.
model_folder = '/content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_java/'
tokenizer_folder = '/content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_java_Tokenizer/'

!unzip '/content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_java/best_model_huawei-noah_TinyBERT_General_4L_312D_java.zip' -d './'
!unzip '/content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_java_Tokenizer/best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer.zip' -d './'

Archive:  /content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_java/best_model_huawei-noah_TinyBERT_General_4L_312D_java.zip
replace ./best_model_huawei-noah_TinyBERT_General_4L_312D_java/training_args.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_huawei-noah_TinyBERT_General_4L_312D_java/training_args.bin  
  inflating: ./best_model_huawei-noah_TinyBERT_General_4L_312D_java/pytorch_model.bin  
  inflating: ./best_model_huawei-noah_TinyBERT_General_4L_312D_java/config.json  
Archive:  /content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_java_Tokenizer/best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer.zip
replace ./best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer/tokenizer_config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer/tokenizer_config.json  
  inflating: ./best_model_huawei-noah_TinyBERT_General_4L_31

In [28]:
best_model_huawei_noah_TinyBERT_General_4L_312D_java = 'best_model_huawei-noah_TinyBERT_General_4L_312D_java'
best_model_huawei_noah_TinyBERT_General_4L_312D_java_tokenizer = 'best_model_huawei-noah_TinyBERT_General_4L_312D_java_tokenizer'

In [29]:
# Load the model and tokenizer from the checkpoint
model = AutoModelForSequenceClassification.from_pretrained(best_model_huawei_noah_TinyBERT_General_4L_312D_java)
tokenizer = AutoTokenizer.from_pretrained(best_model_huawei_noah_TinyBERT_General_4L_312D_java_tokenizer)

loading configuration file best_model_huawei-noah_TinyBERT_General_4L_312D_java/config.json
Model config BertConfig {
  "_name_or_path": "best_model_huawei-noah_TinyBERT_General_4L_312D_java",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "classifier_dropout": null,
  "emb_size": 312,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1200,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
 

In [30]:
text = "equals the @code match field. | AbstractContra...."
encoded_input = tokenizer(text, return_tensors='pt')
start_time = time.time()
output = model(**encoded_input)
end_time = time.time()
prediction_time = end_time - start_time
output

print("=============Prediction Time==============")
print(prediction_time)
print("==========================================")

=============Prediction Time==============
0.023012399673461914


In [31]:
logits = output.logits

# Apply sigmoid to convert logits to probabilities
probabilities = torch.sigmoid(logits)

# Define a threshold to determine if a class is positive
threshold = 0.5
predicted_indices = (probabilities > threshold).nonzero(as_tuple=True)[1]

# Convert indices to a list for output
predicted_classes = (predicted_indices + 1).tolist()

# Format the output to display classes
if len(predicted_classes) > 0:
    predicted_classes_str = ", ".join(map(str, predicted_classes))
    print(f"Predicted class/es: {predicted_classes_str}")
else:
    print("No positive classes predicted.")

Predicted class/es: 3
